In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV

In [54]:
class LeNet(nn.Module): 
    def __init__(self): 
        super(LeNet, self).__init__() 
        self.conv1 = nn.Sequential( 
            nn.Conv2d(1, 6, 5), 
            nn.LeakyReLU(),  
            nn.MaxPool2d(kernel_size=2, stride=2)
        ) 
        self.conv2 = nn.Sequential( 
            nn.Conv2d(6, 16, 5), 
            nn.LeakyReLU(), 
            nn.MaxPool2d(2, 2)
        )
        self.flat = nn.Flatten()
        self.fc1 = nn.Sequential(nn.Linear(16*4*4, 120), nn.LeakyReLU())
        self.fc2 = nn.Sequential(nn.Linear(120, 84), nn.LeakyReLU()) 
        self.fc3 = nn.Linear(84, 10) 

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flat(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [55]:
# 定义全局变量
n_epochs = 10
batch_size = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 加载数据
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
# 创建加载器
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, num_workers = 8)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, num_workers = 8)
# 加载另一种数据
train_loader_all = torch.utils.data.DataLoader(train_data, batch_size = 60000, num_workers = 8)
test_loader_all = torch.utils.data.DataLoader(test_data, batch_size = 60000, num_workers = 8)
for images, labels in train_loader_all:
    X_train, y_train = images, labels
for images, labels in test_loader_all:
    X_test, y_test = images, labels

In [56]:
def test(model):
    correct = 0
    total = 0
    with torch.no_grad(): # 训练集中不需要反向传播
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the test images: %4f %%' % (100 * correct / total))
    return 100.0 * correct / total

In [57]:
def train(model):
    # 定义损失函数和优化器
    lossfunc = nn.CrossEntropyLoss()
    optimizer = optim.SGD(params = model.parameters(), lr = 0.01)
    # 开始训练
    for epoch in range(n_epochs):
        train_loss = 0.0
        for data,target in train_loader:
            data = data.to(device)
            target = target.to(device)
            optimizer.zero_grad() # 清空上一步的残余更新参数值
            output = model(data) # 得到预测值
            loss = lossfunc(output,target) # 计算两者的误差
            loss.backward() # 误差反向传播, 计算参数更新值
            optimizer.step() # 将参数更新值施加到 net 的 parameters 上
            train_loss += loss.item()*data.size(0)
        train_loss = train_loss / len(train_loader.dataset)
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch + 1, train_loss))
        # torch.save(model.state_dict(), f'./weights/bs_20/epoch{epoch + 1}.pth')
        test(model)


In [58]:
# 网络实例化
model = LeNet().to(device)
# 开始训练
if __name__ == '__main__':
    train(model)

Epoch: 1 	Training Loss: 0.954719
Accuracy of the network on the test images: 92.766667 %
Epoch: 2 	Training Loss: 0.137310
Accuracy of the network on the test images: 95.763333 %
Epoch: 3 	Training Loss: 0.091964
Accuracy of the network on the test images: 96.521667 %
Epoch: 4 	Training Loss: 0.073282
Accuracy of the network on the test images: 97.130000 %
Epoch: 5 	Training Loss: 0.061657
Accuracy of the network on the test images: 97.615000 %
Epoch: 6 	Training Loss: 0.053668
Accuracy of the network on the test images: 97.781667 %
Epoch: 7 	Training Loss: 0.047546
Accuracy of the network on the test images: 97.950000 %
Epoch: 8 	Training Loss: 0.042694
Accuracy of the network on the test images: 98.038333 %
Epoch: 9 	Training Loss: 0.038549
Accuracy of the network on the test images: 98.283333 %
Epoch: 10 	Training Loss: 0.034993
Accuracy of the network on the test images: 98.460000 %


In [51]:
def train_CV_1(model):
    model_CV = NeuralNetClassifier(
                model,
                criterion=nn.CrossEntropyLoss,
                optimizer=optim.SGD,
                verbose=False
            )
    param_grid = {
            'batch_size': [10, 20, 40],
            'max_epochs': [10, 15, 20]
            }
    grid = GridSearchCV(estimator=model_CV, param_grid=param_grid, n_jobs=-1, cv=3)
    grid_result = grid.fit(X_train, y_train)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    print("Test: %f" % grid.score(X_test, y_test))

In [52]:
# 网络实例化
model = LeNet().to(device)
# 开始训练
if __name__ == '__main__':
    # train(model)
    train_CV_1(model)

Best: 0.982333 using {'batch_size': 10, 'max_epochs': 20}
Test: 0.991567


In [60]:
def train_CV_2(model):
    model_CV = NeuralNetClassifier(
                model,
                criterion=nn.CrossEntropyLoss,
                optimizer=optim.SGD,
                batch_size=10,
                max_epochs=20,
                verbose=False
            )
    param_grid = {
            'optimizer__lr': [0.001, 0.005, 0.01, 0.02],
            'optimizer__momentum': [0.0, 0.2, 0.4, 0.8],
            }
    grid = GridSearchCV(estimator=model_CV, param_grid=param_grid, n_jobs=-1, cv=3)
    grid_result = grid.fit(X_train, y_train)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    print("Test: %f" % grid.score(X_test, y_test))

In [61]:
# 网络实例化
model = LeNet().to(device)
# 开始训练
if __name__ == '__main__':
    # train(model)
    train_CV_2(model)

Best: 0.984300 using {'optimizer__lr': 0.01, 'optimizer__momentum': 0.8}
Test: 0.994917


# 